In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Universe Rank (252d return) + Rebalance (EMA/RSI overlay) — Groww costs
v2: entry lookback + optional opportunistic onboarding between rebalances.

Selection & Rebalance (every N trading days):
  1) Rank universe by plain 252d return (Close_t / Close_{t-252} - 1)
  2) Keep only stocks with Close >= THR * 52w high (THR=0.50 by default)
  3) Desired set = Top K by 252d return among those passing the 52w filter
  4) Exit held names with rank > DROP_RANK unless 'strong technicals'
     (EMA_fast > EMA_slow AND RSI <= overbought)
  5) Equal-weight across intended set (kept + new entries that satisfy the
     entry condition with lookback) -> trades at next day's open

Daily Technical Overlay:
  - Entry on day t: EMA_fast > EMA_slow AND RSI_{t-1} < oversold AND RSI_t >= oversold
    -> execute at t+1 open, ATR-based SL/TP
  - We also accept entries if the cross happened within the last
    ENTRY_CROSS_LOOKBACK days (and uptrend holds today).
  - Exits any day:
      (a) Intraday SL/TP using High/Low (SL precedence if both)
      (b) RSI > overbought (close exit)

Optional: Opportunistic onboarding between rebalances for names in the current
Desired Top-K set (ENTRY_ON_REBALANCE_ONLY=False).

Outputs:
  - trades.csv (with reasons)
  - equity.csv
"""

from __future__ import annotations
import os, math, json, warnings, logging
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional, Set
from datetime import datetime

import numpy as np
import pandas as pd
import yfinance as yf

warnings.filterwarnings("ignore", category=FutureWarning)

# =========================
# CONFIG
# =========================
CONFIG = {
    # Universe (Yahoo symbols; add .NS for NSE)
        "TICKERS": ['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABLBL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AEGISVOPAK.NS', 'AFCONS.NS', 'AFFLE.NS', 'AGARWALEYE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'AKZOINDIA.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATHERENERG.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', 'BASF.NS', 'BATAINDIA.NS', 'BAYERCROP.NS', 'BBTC.NS', 'BDL.NS', 'BEL.NS', 'BEML.NS', 'BERGEPAINT.NS', 'BHARATFORG.NS', 'BHARTIARTL.NS', 'BHARTIHEXA.NS', 'BHEL.NS', 'BIKAJI.NS', 'BIOCON.NS', 'BLS.NS', 'BLUEDART.NS', 'BLUEJET.NS', 'BLUESTARCO.NS', 'BOSCHLTD.NS', 'BPCL.NS', 'BRIGADE.NS', 'BRITANNIA.NS', 'BSE.NS', 'BSOFT.NS', 'CAMPUS.NS', 'CAMS.NS', 'CANBK.NS', 'CANFINHOME.NS', 'CAPLIPOINT.NS', 'CARBORUNIV.NS', 'CASTROLIND.NS', 'CCL.NS', 'CDSL.NS', 'CEATLTD.NS', 'CENTRALBK.NS', 'CENTURYPLY.NS', 'CERA.NS', 'CESC.NS', 'CGCL.NS', 'CGPOWER.NS', 'CHALET.NS', 'CHAMBLFERT.NS', 'CHENNPETRO.NS', 'CHOICEIN.NS', 'CHOLAFIN.NS', 'CHOLAHLDNG.NS', 'CIPLA.NS', 'CLEAN.NS', 'COALINDIA.NS', 'COCHINSHIP.NS', 'COFORGE.NS', 'COHANCE.NS', 'COLPAL.NS', 'CONCOR.NS', 'CONCORDBIO.NS', 'COROMANDEL.NS', 'CRAFTSMAN.NS', 'CREDITACC.NS', 'CRISIL.NS', 'CROMPTON.NS', 'CUB.NS', 'CUMMINSIND.NS', 'CYIENT.NS', 'DABUR.NS', 'DALBHARAT.NS', 'DATAPATTNS.NS', 'DBREALTY.NS', 'DCMSHRIRAM.NS', 'DEEPAKFERT.NS', 'DEEPAKNTR.NS', 'DELHIVERY.NS', 'DEVYANI.NS', 'DIVISLAB.NS', 'DIXON.NS', 'DLF.NS', 'DMART.NS', 'DOMS.NS', 'DRREDDY.NS', 'ECLERX.NS', 'EICHERMOT.NS', 'EIDPARRY.NS', 'EIHOTEL.NS', 'ELECON.NS', 'ELGIEQUIP.NS', 'EMAMILTD.NS', 'EMCURE.NS', 'ENDURANCE.NS', 'ENGINERSIN.NS', 'ENRIN.NS', 'ERIS.NS', 'ESCORTS.NS', 'ETERNAL.NS', 'EXIDEIND.NS', 'FACT.NS', 'FEDERALBNK.NS', 'FINCABLES.NS', 'FINPIPE.NS', 'FIRSTCRY.NS', 'FIVESTAR.NS', 'FLUOROCHEM.NS', 'FORCEMOT.NS', 'FORTIS.NS', 'FSL.NS', 'GAIL.NS', 'GESHIP.NS', 'GICRE.NS', 'GILLETTE.NS', 'GLAND.NS', 'GLAXO.NS', 'GLENMARK.NS', 'GMDCLTD.NS', 'GMRAIRPORT.NS', 'GODFRYPHLP.NS', 'GODIGIT.NS', 'GODREJAGRO.NS', 'GODREJCP.NS', 'GODREJIND.NS', 'GODREJPROP.NS', 'GPIL.NS', 'GRANULES.NS', 'GRAPHITE.NS', 'GRASIM.NS', 'GRAVITA.NS', 'GRSE.NS', 'GSPL.NS', 'GUJGASLTD.NS', 'GVT&D.NS', 'HAL.NS', 'HAPPSTMNDS.NS', 'HAVELLS.NS', 'HBLENGINE.NS', 'HCLTECH.NS', 'HDFCAMC.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEG.NS', 'HEROMOTOCO.NS', 'HEXT.NS', 'HFCL.NS', 'HINDALCO.NS', 'HINDCOPPER.NS', 'HINDPETRO.NS', 'HINDUNILVR.NS', 'HINDZINC.NS', 'HOMEFIRST.NS', 'HONASA.NS', 'HONAUT.NS', 'HSCL.NS', 'HUDCO.NS', 'HYUNDAI.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'ICICIPRULI.NS', 'IDBI.NS', 'IDEA.NS', 'IDFCFIRSTB.NS', 'IEX.NS', 'IFCI.NS', 'IGIL.NS', 'IGL.NS', 'IIFL.NS', 'IKS.NS', 'INDGN.NS', 'INDHOTEL.NS', 'INDIACEM.NS', 'INDIAMART.NS', 'INDIANB.NS', 'INDIGO.NS', 'INDUSINDBK.NS', 'INDUSTOWER.NS', 'INFY.NS', 'INOXINDIA.NS', 'INOXWIND.NS', 'INTELLECT.NS', 'IOB.NS', 'IOC.NS', 'IPCALAB.NS', 'IRB.NS', 'IRCON.NS', 'IRCTC.NS', 'IREDA.NS', 'IRFC.NS', 'ITC.NS', 'ITCHOTELS.NS', 'ITI.NS', 'J&KBANK.NS', 'JBCHEPHARM.NS', 'JBMA.NS', 'JINDALSAW.NS', 'JINDALSTEL.NS', 'JIOFIN.NS', 'JKCEMENT.NS', 'JKTYRE.NS', 'JMFINANCIL.NS', 'JPPOWER.NS', 'JSL.NS', 'JSWENERGY.NS', 'JSWINFRA.NS', 'JSWSTEEL.NS', 'JUBLFOOD.NS', 'JUBLINGREA.NS', 'JUBLPHARMA.NS', 'JWL.NS', 'JYOTHYLAB.NS', 'JYOTICNC.NS', 'KAJARIACER.NS', 'KALYANKJIL.NS', 'KARURVYSYA.NS', 'KAYNES.NS', 'KEC.NS', 'KEI.NS', 'KFINTECH.NS', 'KIMS.NS', 'KIRLOSBROS.NS', 'KIRLOSENG.NS', 'KOTAKBANK.NS', 'KPIL.NS', 'KPITTECH.NS', 'KPRMILL.NS', 'KSB.NS', 'LALPATHLAB.NS', 'LATENTVIEW.NS', 'LAURUSLABS.NS', 'LEMONTREE.NS', 'LICHSGFIN.NS', 'LICI.NS', 'LINDEINDIA.NS', 'LLOYDSME.NS', 'LODHA.NS', 'LT.NS', 'LTF.NS', 'LTFOODS.NS', 'LTIM.NS', 'LTTS.NS', 'LUPIN.NS', 'M&M.NS', 'M&MFIN.NS', 'MAHABANK.NS', 'MAHSCOOTER.NS', 'MAHSEAMLES.NS', 'MANAPPURAM.NS', 'MANKIND.NS', 'MANYAVAR.NS', 'MAPMYINDIA.NS', 'MARICO.NS', 'MARUTI.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MCX.NS', 'MEDANTA.NS', 'METROPOLIS.NS', 'MFSL.NS', 'MGL.NS', 'MINDACORP.NS', 'MMTC.NS', 'MOTHERSON.NS', 'MOTILALOFS.NS', 'MPHASIS.NS', 'MRF.NS', 'MRPL.NS', 'MSUMI.NS', 'MUTHOOTFIN.NS', 'NAM-INDIA.NS', 'NATCOPHARM.NS', 'NATIONALUM.NS', 'NAUKRI.NS', 'NAVA.NS', 'NAVINFLUOR.NS', 'NBCC.NS', 'NCC.NS', 'NESTLEIND.NS', 'NETWEB.NS', 'NEULANDLAB.NS', 'NEWGEN.NS', 'NH.NS', 'NHPC.NS', 'NIACL.NS', 'NIVABUPA.NS', 'NLCINDIA.NS', 'NMDC.NS', 'NSLNISP.NS', 'NTPC.NS', 'NTPCGREEN.NS', 'NUVAMA.NS', 'NUVOCO.NS', 'NYKAA.NS', 'OBEROIRLTY.NS', 'OFSS.NS', 'OIL.NS', 'OLAELEC.NS', 'OLECTRA.NS', 'ONESOURCE.NS', 'ONGC.NS', 'PAGEIND.NS', 'PATANJALI.NS', 'PAYTM.NS', 'PCBL.NS', 'PERSISTENT.NS', 'PETRONET.NS', 'PFC.NS', 'PFIZER.NS', 'PGEL.NS', 'PGHH.NS', 'PHOENIXLTD.NS', 'PIDILITIND.NS', 'PIIND.NS', 'PNB.NS', 'PNBHOUSING.NS', 'POLICYBZR.NS', 'POLYCAB.NS', 'POLYMED.NS', 'POONAWALLA.NS', 'POWERGRID.NS', 'POWERINDIA.NS', 'PPLPHARMA.NS', 'PRAJIND.NS', 'PREMIERENE.NS', 'PRESTIGE.NS', 'PTCIL.NS', 'PVRINOX.NS', 'RADICO.NS', 'RAILTEL.NS', 'RAINBOW.NS', 'RAMCOCEM.NS', 'RBLBANK.NS', 'RCF.NS', 'RECLTD.NS', 'REDINGTON.NS', 'RELIANCE.NS', 'RELINFRA.NS', 'RHIM.NS', 'RITES.NS', 'RKFORGE.NS', 'RPOWER.NS', 'RRKABEL.NS', 'RVNL.NS', 'SAGILITY.NS', 'SAIL.NS', 'SAILIFE.NS', 'SAMMAANCAP.NS', 'SAPPHIRE.NS', 'SARDAEN.NS', 'SAREGAMA.NS', 'SBFC.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SBIN.NS', 'SCHAEFFLER.NS', 'SCHNEIDER.NS', 'SCI.NS', 'SHREECEM.NS', 'SHRIRAMFIN.NS', 'SHYAMMETL.NS', 'SIEMENS.NS', 'SIGNATURE.NS', 'SJVN.NS', 'SKFINDIA.NS', 'SOBHA.NS', 'SOLARINDS.NS', 'SONACOMS.NS', 'SONATSOFTW.NS', 'SRF.NS', 'STARHEALTH.NS', 'SUMICHEM.NS', 'SUNDARMFIN.NS', 'SUNDRMFAST.NS', 'SUNPHARMA.NS', 'SUNTV.NS', 'SUPREMEIND.NS', 'SUZLON.NS', 'SWANCORP.NS', 'SWIGGY.NS', 'SYNGENE.NS', 'SYRMA.NS', 'TARIL.NS', 'TATACHEM.NS', 'TATACOMM.NS', 'TATACONSUM.NS', 'TATAELXSI.NS', 'TATAINVEST.NS', 'TATAMOTORS.NS', 'TATAPOWER.NS', 'TATASTEEL.NS', 'TATATECH.NS', 'TBOTEK.NS', 'TCS.NS', 'TECHM.NS', 'TECHNOE.NS', 'TEJASNET.NS', 'THELEELA.NS', 'THERMAX.NS', 'TIINDIA.NS', 'TIMKEN.NS', 'TITAGARH.NS', 'TITAN.NS', 'TORNTPHARM.NS', 'TORNTPOWER.NS', 'TRENT.NS', 'TRIDENT.NS', 'TRITURBINE.NS', 'TRIVENI.NS', 'TTML.NS', 'TVSMOTOR.NS', 'UBL.NS', 'UCOBANK.NS', 'ULTRACEMCO.NS', 'UNIONBANK.NS', 'UNITDSPR.NS', 'UNOMINDA.NS', 'UPL.NS', 'USHAMART.NS', 'UTIAMC.NS', 'VBL.NS', 'VEDL.NS', 'VENTIVE.NS', 'VGUARD.NS', 'VIJAYA.NS', 'VMM.NS', 'VOLTAS.NS', 'VTL.NS', 'WAAREEENER.NS', 'WELCORP.NS', 'WELSPUNLIV.NS', 'WHIRLPOOL.NS', 'WIPRO.NS', 'WOCKPHARMA.NS', 'YESBANK.NS', 'ZEEL.NS', 'ZENSARTECH.NS', 'ZENTEC.NS', 'ZFCVINDIA.NS', 'ZYDUSLIFE.NS'],


    # Calendar source
    "BENCH_TICKER": "^NSEI",        # NIFTY 50 calendar is fine
    "START_DATE": "2010-01-01",
    "END_DATE": None,               # None -> today

    # Cache
    "CACHE_DIR": "./cache_yf",
    "FORCE_REFRESH": False,

    # Technical parameters
    "EMA_FAST": 20,
    "EMA_SLOW": 50,
    "RSI_LEN": 14,
    "RSI_OVERSOLD": 30.0,
    "RSI_OVERBOUGHT": 70.0,
    "ATR_LEN": 14,
    "SL_ATR_MULT": 1.5,
    "TARGET_R": 2.0,

    # Entry gating improvements
    "ENTRY_CROSS_LOOKBACK": 10,         # accept RSI cross within last N days (incl. today)
    "ENTRY_ON_REBALANCE_ONLY": False,   # if False, also onboard between rebalances for current Top-K

    # Rebalance / selection settings
    "REBAL_EVERY_N_DAYS": 20,           # trading days
    "TOP_K": 5,
    "DROP_RANK_THRESHOLD": 40,          # if current holding rank > 40 -> exit unless strong techs
    "NEAR_52W_HIGH_THRESHOLD": 0.50,    # Close >= 0.5 * 52w high

    # Portfolio / execution
    "START_CAPITAL_INR": 1_000_000.0,
    "SLIPPAGE_BPS": 0,
    "APPLY_FEES": True,
    "LOT_SIZE": 1,                      # integers; keep 1 unless you need round lots

    # Logging
    "LOG_LEVEL": "INFO",
}

if CONFIG["END_DATE"] is None:
    CONFIG["END_DATE"] = datetime.now().date().isoformat()

# =========================
# Logging
# =========================
def setup_logging(level="INFO"):
    lvl = getattr(logging, level.upper(), logging.INFO)
    logging.basicConfig(
        level=lvl, format="%(asctime)s | %(levelname)s | %(message)s", datefmt="%H:%M:%S"
    )

# =========================
# Groww cost (given)
# =========================
def calc_fees(turnover_buy: float, turnover_sell: float) -> float:
    if not CONFIG["APPLY_FEES"]: return 0.0
    BROKER_PCT = 0.001; BROKER_MIN = 5.0; BROKER_CAP = 20.0
    STT_PCT = 0.001; STAMP_BUY_PCT = 0.00015
    EXCH_PCT = 0.0000297; SEBI_PCT = 0.000001; IPFT_PCT = 0.000001
    GST_PCT = 0.18; DP_SELL = 20.0 if turnover_sell >= 100 else 0.0
    def _broker(turn): return max(BROKER_MIN, min(turn*BROKER_PCT, BROKER_CAP)) if turn>0 else 0
    brb, brs = _broker(turnover_buy), _broker(turnover_sell)
    stt = STT_PCT*(turnover_buy+turnover_sell)
    stamp = STAMP_BUY_PCT*turnover_buy
    exch = EXCH_PCT*(turnover_buy+turnover_sell)
    sebi = SEBI_PCT*(turnover_buy+turnover_sell)
    ipft = IPFT_PCT*(turnover_buy+turnover_sell)
    dp = DP_SELL
    gst = GST_PCT*(brb+brs+dp+exch+sebi+ipft)
    return float((brb+brs)+stt+stamp+exch+sebi+ipft+dp+gst)

# =========================
# Data utils
# =========================
def ensure_dir(p: str): os.makedirs(p, exist_ok=True)

def to_ist_date_index(df: pd.DataFrame) -> pd.DataFrame:
    if not isinstance(df.index, pd.DatetimeIndex): return df
    df = df.copy()
    if df.index.tz is None:
        df.index = df.index.tz_localize("UTC").tz_convert("Asia/Kolkata")
    else:
        df.index = df.index.tz_convert("Asia/Kolkata")
    df.index = pd.to_datetime(df.index.date)
    return df

def load_yf(ticker: str, start: str, end: str, cache_dir: str, force_refresh: bool) -> pd.DataFrame:
    ensure_dir(cache_dir)
    safe = ticker.replace("^", "_")
    path = os.path.join(cache_dir, f"{safe}.csv")

    def _clean(df0: pd.DataFrame) -> pd.DataFrame:
        if df0.empty: return df0
        cols = [c for c in ["Open","High","Low","Close","Adj Close","Volume"] if c in df0.columns]
        df = df0[cols].copy()
        if "Adj Close" not in df and "Close" in df: df["Adj Close"] = df["Close"]
        df = to_ist_date_index(df)
        df = df[~df.index.duplicated(keep="first")].sort_index()
        return df

    if (not force_refresh) and os.path.exists(path):
        try:
            df = pd.read_csv(path, parse_dates=["Date"]).set_index("Date")
            df = _clean(df)
            if not df.empty:
                return df
        except Exception:
            pass

    df = yf.download(ticker, start=start, end=end, interval="1d", auto_adjust=False, progress=False, multi_level_index=False)
    if df.empty:
        raise ValueError(f"No data for {ticker}")
    df = _clean(df)
    df.to_csv(path, index_label="Date")
    return df

# =========================
# Indicators
# =========================
def ema(s: pd.Series, span: int) -> pd.Series:
    return s.ewm(span=span, adjust=False).mean()

def rsi_wilder(close: pd.Series, length: int = 14) -> pd.Series:
    delta = close.diff()
    up = delta.clip(lower=0.0)
    down = -delta.clip(upper=0.0)
    roll_up = up.ewm(alpha=1/length, adjust=False).mean()
    roll_down = down.ewm(alpha=1/length, adjust=False).mean()
    rs = roll_up / (roll_down.replace(0, np.nan))
    return (100 - (100 / (1 + rs))).fillna(50.0)

def atr_wilder(high: pd.Series, low: pd.Series, close: pd.Series, length=14) -> pd.Series:
    prev_close = close.shift(1)
    tr = pd.concat([(high - low), (high - prev_close).abs(), (low - prev_close).abs()], axis=1).max(axis=1)
    return tr.ewm(alpha=1/length, adjust=False).mean()

# =========================
# Signals (EMA + RSI with lookback)
# =========================
def build_signals(df: pd.DataFrame, p: Dict) -> pd.DataFrame:
    o,h,l,c,v = df["Open"], df["High"], df["Low"], df["Close"], df["Volume"]
    ef, es = ema(c, p["EMA_FAST"]), ema(c, p["EMA_SLOW"])
    r = rsi_wilder(c, p["RSI_LEN"])
    atr = atr_wilder(h, l, c, p["ATR_LEN"])

    entry_cross = ( (r.shift(1) < p["RSI_OVERSOLD"]) & (r >= p["RSI_OVERSOLD"]) )
    uptrend = ef > es
    entry_today = (uptrend & entry_cross).astype(int)

    # Accept a recent cross within lookback if still in uptrend today
    lb = int(CONFIG.get("ENTRY_CROSS_LOOKBACK", 0) or 0)
    if lb > 1:
        entry_recent = ((entry_cross.rolling(lb).max() == 1) & uptrend).astype(int)
    else:
        entry_recent = entry_today.copy()

    rsi_ob = (r > p["RSI_OVERBOUGHT"]).astype(int)

    return pd.DataFrame({
        "Open": o, "High": h, "Low": l, "Close": c, "Volume": v,
        "ema_fast": ef, "ema_slow": es, "rsi": r, "atr": atr,
        "entry_today": entry_today,
        "entry_recent": entry_recent,
        "exit_rsi_ob": rsi_ob
    }, index=df.index)

# =========================
# Ranking helpers (252d return + 52w filter)
# =========================
def ret252_on_day(close: pd.Series, day: pd.Timestamp) -> Optional[float]:
    if day not in close.index: return None
    idx = close.index.get_loc(day)
    if isinstance(idx, (np.ndarray, list)): idx = int(idx[0])
    if idx < 252: return None
    prev = close.iloc[idx-252]
    today = close.iloc[idx]
    if pd.isna(prev) or prev == 0 or pd.isna(today): return None
    return float(today/prev - 1.0)

def rank_by_252d_return_and_52w_filter(
    day: pd.Timestamp,
    sig_map: Dict[str,pd.DataFrame],
    top_k: int,
    near_high_thr: float
) -> pd.DataFrame:
    rows = []
    for tkr, s in sig_map.items():
        if day not in s.index: continue
        r252 = ret252_on_day(s["Close"], day)
        if r252 is None: continue
        high_52w = s["Close"].rolling(252).max().loc[day]
        close = s.at[day, "Close"]
        pass_52w = (close >= near_high_thr * high_52w) if (pd.notna(high_52w) and high_52w > 0) else False
        if not pass_52w: continue
        rows.append((tkr, r252, close, high_52w, pass_52w))
    if not rows:
        return pd.DataFrame(columns=["ticker","rank","ret252","close","high_52w","pass_52w"])
    df = pd.DataFrame(rows, columns=["ticker","ret252","close","high_52w","pass_52w"])
    df = df.sort_values("ret252", ascending=False).reset_index(drop=True)
    df["rank"] = np.arange(1, len(df)+1)
    return df[["ticker","rank","ret252","close","high_52w","pass_52w"]]

# =========================
# Position model
# =========================
@dataclass
class Position:
    ticker: str
    shares: int
    entry_price: float
    sl: float
    tp: float

# =========================
# Portfolio Simulator
# =========================
def _slip(px: float) -> float:
    bps = CONFIG["SLIPPAGE_BPS"]
    return px * (1 + (bps/10000.0)) if bps != 0 else px

def simulate_portfolio(sig_map: Dict[str,pd.DataFrame], bench: pd.DataFrame, p: Dict) -> Tuple[pd.DataFrame, pd.DataFrame]:
    cal = bench.index
    cash = CONFIG["START_CAPITAL_INR"]
    holdings: Dict[str, Position] = {}
    equity = pd.Series(index=cal, dtype=float)
    trades: List[Dict] = []

    next_open_orders: Dict[pd.Timestamp, List[Dict]] = {}
    pending_equalize: Dict[pd.Timestamp, Set[str]] = {}

    reb_every = CONFIG["REBAL_EVERY_N_DAYS"]
    top_k = CONFIG["TOP_K"]
    drop_rank_thr = CONFIG["DROP_RANK_THRESHOLD"]
    near_thr = CONFIG["NEAR_52W_HIGH_THRESHOLD"]
    entry_on_rebal_only = bool(CONFIG.get("ENTRY_ON_REBALANCE_ONLY", True))

    current_desired: List[str] = []  # persist between rebalances

    for i, d in enumerate(cal):
        # 1) Execute next-open orders (SELLs then BUYs)
        if d in next_open_orders:
            # Sells first
            for ord_ in [o for o in next_open_orders[d] if o["side"]=="SELL"]:
                tkr = ord_["ticker"]
                if tkr not in sig_map or d not in sig_map[tkr].index: 
                    continue
                oprice = _slip(sig_map[tkr].at[d, "Open"])
                shares = min(ord_.get("shares", 0), holdings.get(tkr, Position(tkr,0,0,0,0)).shares)
                if shares <= 0: 
                    continue
                sell_turn = oprice * shares
                fee = calc_fees(0.0, sell_turn)
                cash += sell_turn - fee
                pos = holdings.get(tkr)
                if pos:
                    pos.shares -= shares
                    reason = ord_.get("reason", "rebalance_sell")
                    pnl = (oprice - pos.entry_price) * shares - fee
                    trades.append(dict(ticker=tkr, side="SELL", date=d, price=oprice, shares=shares, reason=reason, pnl=pnl))
                    if pos.shares <= 0:
                        holdings.pop(tkr, None)
            # Buys
            for ord_ in [o for o in next_open_orders[d] if o["side"]=="BUY"]:
                tkr = ord_["ticker"]
                if tkr not in sig_map or d not in sig_map[tkr].index: 
                    continue
                oprice = _slip(sig_map[tkr].at[d, "Open"])
                if oprice <= 0:
                    continue
                desired_value = float(ord_.get("target_value", 0.0))
                if desired_value > 0:
                    shares = int(max(0, (desired_value // (oprice * CONFIG["LOT_SIZE"])) * CONFIG["LOT_SIZE"]))
                else:
                    shares = int(ord_.get("shares", 0))
                if shares <= 0:
                    continue
                cost = oprice * shares
                fee = calc_fees(cost, 0.0)
                total = cost + fee
                if total > cash + 1e-9:
                    max_sh = int((cash // (oprice * CONFIG["LOT_SIZE"])) * CONFIG["LOT_SIZE"])
                    shares = max(0, max_sh)
                    if shares <= 0:
                        continue
                    cost = oprice * shares
                    fee = calc_fees(cost, 0.0)
                    total = cost + fee
                cash -= total
                atr = sig_map[tkr].at[d, "atr"]
                sl = oprice - p["SL_ATR_MULT"]*atr
                tp = oprice + p["TARGET_R"]*(p["SL_ATR_MULT"]*atr)
                if tkr in holdings:
                    pos = holdings[tkr]
                    total_shares = pos.shares + shares
                    avg_entry = (pos.entry_price*pos.shares + oprice*shares) / total_shares
                    pos.shares = total_shares
                    pos.entry_price = avg_entry
                    pos.sl = sl; pos.tp = tp
                else:
                    holdings[tkr] = Position(tkr, shares, oprice, sl, tp)
                trades.append(dict(ticker=tkr, side="BUY", date=d, price=oprice, shares=shares, reason=ord_.get("reason","rebalance_buy"), pnl=0.0))
            next_open_orders.pop(d, None)

        # 2) Intraday SL/TP (SL precedence)
        to_remove = []
        for tkr, pos in list(holdings.items()):
            s = sig_map.get(tkr)
            if s is None or d not in s.index:
                continue
            h, l = s.at[d,"High"], s.at[d,"Low"]
            sl_hit = l <= pos.sl
            tp_hit = h >= pos.tp
            if sl_hit or tp_hit:
                exit_px = _slip(pos.sl if sl_hit else pos.tp)
                proceeds = exit_px * pos.shares
                fee = calc_fees(0.0, proceeds)
                pnl = proceeds - fee - (pos.entry_price * pos.shares)
                cash += proceeds - fee
                trades.append(dict(ticker=tkr, side="SELL", date=d, price=exit_px, shares=pos.shares,
                                   reason="exit_stop" if sl_hit else "exit_target", pnl=pnl))
                to_remove.append(tkr)
        for t in to_remove: holdings.pop(t, None)

        # 3) Close-based RSI>OB exits
        to_remove2 = []
        for tkr, pos in list(holdings.items()):
            s = sig_map.get(tkr)
            if s is None or d not in s.index: continue
            if s.at[d,"exit_rsi_ob"] == 1:
                close_px = _slip(s.at[d,"Close"])
                proceeds = close_px * pos.shares
                fee = calc_fees(0.0, proceeds)
                pnl = proceeds - fee - (pos.entry_price * pos.shares)
                cash += proceeds - fee
                trades.append(dict(ticker=tkr, side="SELL", date=d, price=close_px, shares=pos.shares,
                                   reason="exit_rsi_overbought", pnl=pnl))
                to_remove2.append(tkr)
        for t in to_remove2: holdings.pop(t, None)

        # 4) Rebalance scheduling (every N trading days)
        is_rebal = (i >= 252) and (reb_every > 0) and ((i % reb_every) == 0)
        if is_rebal:
            rank_df = rank_by_252d_return_and_52w_filter(d, sig_map, CONFIG["TOP_K"], near_thr)
            desired = rank_df.head(top_k)["ticker"].tolist()
            current_desired = desired[:]  # persist until next rebalance

            logging.info(f"[REBAL {d.date()}] filtered={len(rank_df)} desired={len(desired)} top5={desired}")

            # ranks for drop rule
            rank_map = {r.ticker: int(r.rank) for r in rank_df.itertuples(index=False)}

            def strong_technicals(t: str) -> bool:
                s = sig_map[t]; return bool((s.at[d,"ema_fast"] > s.at[d,"ema_slow"]) and (s.at[d,"rsi"] <= CONFIG["RSI_OVERBOUGHT"]))

            # schedule exits for holdings with rank > threshold unless strong techs
            for tkr in list(holdings.keys()):
                rnk = rank_map.get(tkr, 10_000)
                if rnk > drop_rank_thr and not strong_technicals(tkr):
                    if i + 1 < len(cal):
                        d_next = cal[i+1]
                        next_open_orders.setdefault(d_next, []).append(
                            dict(side="SELL", ticker=tkr, shares=holdings[tkr].shares, reason="rebalance_rank_drop")
                        )

            # schedule buys for desired not held if entry_recent fires today
            d_next = cal[i+1] if (i+1 < len(cal)) else None
            if d_next is not None and len(desired) > 0:
                for tkr in desired:
                    if tkr in holdings: continue
                    s = sig_map.get(tkr)
                    if s is None or d not in s.index: continue
                    if s.at[d,"entry_recent"] == 1:
                        next_open_orders.setdefault(d_next, []).append(
                            dict(side="BUY", ticker=tkr, reason="rebalance_topK_entry")
                        )

                # Equal-weight plan for next open among intended set (kept + planned buys)
                to_sell = {o["ticker"] for o in next_open_orders.get(d_next, []) if o["side"]=="SELL"}
                kept = set(holdings.keys()) - to_sell
                planned_buys = {o["ticker"] for o in next_open_orders.get(d_next, []) if o["side"]=="BUY"}
                intended = list(kept | planned_buys)
                pending_equalize[d_next] = set(intended)

        # 5) Opportunistic onboarding between rebalances (optional)
        if (not is_rebal) and (not entry_on_rebal_only) and current_desired:
            # If a current-desired name is unheld and flashes entry_recent today, schedule a buy for next open
            d_next = cal[i+1] if (i+1 < len(cal)) else None
            if d_next is not None:
                # Estimate total equity now (using close)
                total_equity_est = cash
                for tkr, pos in holdings.items():
                    s = sig_map.get(tkr)
                    if s is not None and d in s.index:
                        total_equity_est += pos.shares * s.at[d,"Close"]
                per_value_est = total_equity_est / max(1, len(current_desired))

                for tkr in current_desired:
                    if tkr in holdings: continue
                    s = sig_map.get(tkr)
                    if s is None or d not in s.index: continue
                    if s.at[d,"entry_recent"] == 1:
                        # target up to per-value
                        target_value = per_value_est
                        next_open_orders.setdefault(d_next, []).append(
                            dict(side="BUY", ticker=tkr, target_value=target_value, reason="opportunistic_entry")
                        )

        # 6) Attach equal-weight target values right before next-open execution
        if d in pending_equalize:
            target_set = pending_equalize.pop(d)
            total_equity = cash
            open_prices = {}
            for tkr in target_set:
                if tkr in sig_map and d in sig_map[tkr].index:
                    opx = sig_map[tkr].at[d, "Open"]
                    open_prices[tkr] = opx
            for tkr, pos in holdings.items():
                if tkr in open_prices:
                    total_equity += pos.shares * open_prices[tkr]
            per_value = total_equity / max(1, len(open_prices)) if open_prices else 0.0

            eq_orders = next_open_orders.setdefault(d, [])
            for tkr, opx in open_prices.items():
                curr_sh = holdings[tkr].shares if tkr in holdings else 0
                curr_val = curr_sh * opx
                delta_val = per_value - curr_val
                if abs(delta_val) < 1e-6: continue
                if delta_val > 0:
                    eq_orders.append(dict(side="BUY", ticker=tkr, target_value=delta_val, reason="rebalance_equalize"))
                else:
                    need_sell_val = -delta_val
                    shares = int(max(0, (need_sell_val // (opx * CONFIG["LOT_SIZE"])) * CONFIG["LOT_SIZE"]))
                    if shares > 0 and tkr in holdings and shares <= holdings[tkr].shares:
                        eq_orders.append(dict(side="SELL", ticker=tkr, shares=shares, reason="rebalance_equalize_trim"))

        # 7) Mark-to-market
        mtm = cash
        for tkr, pos in holdings.items():
            s = sig_map.get(tkr)
            if s is not None and d in s.index:
                mtm += pos.shares * s.at[d, "Close"]
        equity.loc[d] = mtm

    # Force close all at last day close
    if len(cal) > 0:
        d = cal[-1]
        for tkr, pos in list(holdings.items()):
            s = sig_map.get(tkr)
            if s is None or d not in s.index: continue
            px = _slip(s.at[d,"Close"])
            proceeds = px * pos.shares
            fee = calc_fees(0.0, proceeds)
            pnl = proceeds - fee - (pos.entry_price * pos.shares)
            trades.append(dict(ticker=tkr, side="SELL", date=d, price=px, shares=pos.shares, reason="force_close", pnl=pnl))
        holdings.clear()

    trades_df = pd.DataFrame(trades, columns=["ticker","side","date","price","shares","reason","pnl"])
    equity_df = equity.rename("equity").to_frame()
    return trades_df, equity_df

# =========================
# Orchestration
# =========================
def main():
    setup_logging(CONFIG.get("LOG_LEVEL","INFO"))
    print("=== Universe Rank (252d return) + Rebalance (EMA/RSI overlay) — Groww costs v2 ===")
    print(f"Universe size: {len(CONFIG['TICKERS'])}")
    print(f"Window:       {CONFIG['START_DATE']} .. {CONFIG['END_DATE']}")
    print(f"Top-K:        {CONFIG['TOP_K']} | Drop if rank > {CONFIG['DROP_RANK_THRESHOLD']}")
    print(f"Rebalance:    every {CONFIG['REBAL_EVERY_N_DAYS']} trading days")
    print(f"52w filter:   within {int(CONFIG['NEAR_52W_HIGH_THRESHOLD']*100)}% of 52w high")
    print(f"Entry lb:     {CONFIG['ENTRY_CROSS_LOOKBACK']} days | Rebal-only: {CONFIG['ENTRY_ON_REBALANCE_ONLY']}")
    print(f"Start Cap:    ₹{CONFIG['START_CAPITAL_INR']:,} | Fees: {'ON' if CONFIG['APPLY_FEES'] else 'OFF'}")

    start, end = CONFIG["START_DATE"], CONFIG["END_DATE"]
    cache, force = CONFIG["CACHE_DIR"], CONFIG["FORCE_REFRESH"]

    if not CONFIG["TICKERS"]:
        raise ValueError("Please populate CONFIG['TICKERS'] with your universe (e.g., 500 NSE tickers).")

    # Load benchmark calendar
    bench = load_yf(CONFIG["BENCH_TICKER"], start, end, cache, force)

    # Load universe & build signals
    data_map = {}
    for t in CONFIG["TICKERS"]:
        try:
            df = load_yf(t, start, end, cache, force)
            data_map[t] = df
        except Exception as e:
            logging.warning(f"[DATA] {t}: {e}")

    if not data_map:
        raise RuntimeError("No data loaded for any ticker in universe.")

    p = dict(
        EMA_FAST=CONFIG["EMA_FAST"], EMA_SLOW=CONFIG["EMA_SLOW"],
        RSI_LEN=CONFIG["RSI_LEN"], RSI_OVERSOLD=CONFIG["RSI_OVERSOLD"], RSI_OVERBOUGHT=CONFIG["RSI_OVERBOUGHT"],
        ATR_LEN=CONFIG["ATR_LEN"], SL_ATR_MULT=CONFIG["SL_ATR_MULT"], TARGET_R=CONFIG["TARGET_R"]
    )

    sig_map = {t: build_signals(df, p) for t, df in data_map.items()}

    trades, equity = simulate_portfolio(sig_map, bench, p)

    # Save outputs
    if trades.empty:
        logging.info("[RESULT] No trades generated.")
        pd.DataFrame(columns=["ticker","side","date","price","shares","reason","pnl"]).to_csv("trades.csv", index=False)
    else:
        if "date" not in trades.columns and isinstance(trades.index, pd.DatetimeIndex):
            trades = trades.reset_index().rename(columns={"index":"date"})
        trades = trades.sort_values(["date","ticker","side"])
        trades.to_csv("trades.csv", index=False)

    equity.to_csv("equity.csv", index=True)

    # Simple summary
    eq = equity["equity"]
    rets = eq.pct_change().fillna(0.0)
    mean, std = rets.mean(), rets.std(ddof=0)
    sharpe = 0.0 if std==0 else (mean/std)*np.sqrt(252)
    dd = (eq/eq.cummax()-1).min()
    days = (eq.index[-1]-eq.index[0]).days
    cagr = 0.0 if days<=0 else (eq.iloc[-1]/eq.iloc[0])**(365.25/days)-1.0

    summary = dict(
        start_equity=float(eq.iloc[0]),
        end_equity=float(eq.iloc[-1]),
        cagr_pct=100*float(cagr),
        sharpe=float(sharpe),
        max_drawdown_pct=100*float(dd),
        n_trades=int(len(trades)),
    )
    print("\n=== Summary ===")
    print(json.dumps(summary, indent=2))
    print("\nFiles saved:\n  - trades.csv\n  - equity.csv")

if __name__ == "__main__":
    main()


=== Universe Rank (252d return) + Rebalance (EMA/RSI overlay) — Groww costs v2 ===
Universe size: 500
Window:       2010-01-01 .. 2025-10-06
Top-K:        5 | Drop if rank > 40
Rebalance:    every 20 trading days
52w filter:   within 50% of 52w high
Entry lb:     10 days | Rebal-only: False
Start Cap:    ₹1,000,000.0 | Fees: ON


00:42:56 | INFO | [REBAL 2011-01-17] filtered=283 desired=5 top5=['UBL.NS', 'CHOLAFIN.NS', 'TITAN.NS', 'NATCOPHARM.NS', 'COROMANDEL.NS']
00:42:56 | INFO | [REBAL 2011-02-15] filtered=282 desired=5 top5=['UBL.NS', 'CHOLAFIN.NS', 'ECLERX.NS', 'WOCKPHARMA.NS', 'NATCOPHARM.NS']
00:42:56 | INFO | [REBAL 2011-03-16] filtered=281 desired=5 top5=['UBL.NS', 'WOCKPHARMA.NS', 'ELGIEQUIP.NS', 'BAJFINANCE.NS', 'JBCHEPHARM.NS']
00:42:56 | INFO | [REBAL 2011-04-15] filtered=287 desired=5 top5=['WOCKPHARMA.NS', 'UBL.NS', 'ECLERX.NS', 'M&MFIN.NS', 'COROMANDEL.NS']
00:42:56 | INFO | [REBAL 2011-05-16] filtered=284 desired=5 top5=['WOCKPHARMA.NS', 'UBL.NS', 'JUBLFOOD.NS', 'PAGEIND.NS', 'ECLERX.NS']
00:42:57 | INFO | [REBAL 2011-06-14] filtered=281 desired=5 top5=['JUBLFOOD.NS', 'WOCKPHARMA.NS', 'UBL.NS', 'VGUARD.NS', 'ECLERX.NS']
00:42:57 | INFO | [REBAL 2011-07-12] filtered=279 desired=5 top5=['WOCKPHARMA.NS', 'JUBLFOOD.NS', 'BATAINDIA.NS', 'UBL.NS', 'VGUARD.NS']
00:42:57 | INFO | [REBAL 2011-08-10] fil


=== Summary ===
{
  "start_equity": 1000000.0,
  "end_equity": 986470.4617296617,
  "cagr_pct": -0.08647619727195632,
  "sharpe": -0.02854247766458691,
  "max_drawdown_pct": -7.561960905637488,
  "n_trades": 44
}

Files saved:
  - trades.csv
  - equity.csv
